In [21]:
import requests 
from bs4 import BeautifulSoup   
import numpy as np 
import pandas as pd



In [22]:
RANDOM_SEED = 42

In [23]:
test_cars=pd.read_csv('test_cars.csv')


In [24]:
test_cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 39 columns):
bodyType                34686 non-null object
brand                   34686 non-null object
car_url                 34686 non-null object
color                   34686 non-null object
complectation_dict      6418 non-null object
description             34686 non-null object
engineDisplacement      34686 non-null object
enginePower             34686 non-null object
equipment_dict          24694 non-null object
fuelType                34683 non-null object
image                   34686 non-null object
mileage                 34686 non-null object
modelDate               34686 non-null object
model_info              34686 non-null object
model_name              34686 non-null object
name                    34685 non-null object
numberOfDoors           34686 non-null object
parsing_unixtime        34686 non-null object
priceCurrency           34686 non-null object
productionDate    

In [25]:
test_cars=test_cars.drop(['Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38'], axis=1)
test_cars.head(3)

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,...,LIFTBACK ROBOT 1.2,роботизированная,EUROPEAN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,LIFTBACK MECHANICAL 1.6,механическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
2,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/superb/11...,серый,"{""id"":""20026336"",""name"":""Ambition"",""available_...","Все автомобили, представленные в продаже, прох...",1.8 LTR,152 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,...,LIFTBACK ROBOT 1.8,роботизированная,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен


Задаём пустой дата сет, чтоб в него сохранять данные, которые напарсим с сайта

In [27]:
#df_marks_and_models = pd.read_csv ('parcing_from_autoru.csv')


Парсинг

Адрес страницы, которую будем парсить

In [ ]:
url='https://auto.ru/-/ajax/desktop/listing/'

HEADERS - HTML - элемент, представляет собой вводный контент, обычно группу вводных или навигационных средств. Он может содержать другие элементы-заголовки, а также логотип, форму поиска, имя автора и другие элементы.


In [ ]:
HEADERS = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
    'Connection': 'keep-alive',
    
    'content-type': 'application/json',
    'Cookie': '_csrf_token=d8029e40b7e75f491e05b9191bce2b0854cb8560d32b9010; autoru_sid=a%3Ag5fcbae7b2rh31h6p5i54qkso13bmhrs.d64fdb1048094ade6653bb8d77bfc82f%7C1607183995440.604800.lS7lenbdJu7TVJgQ9a2fBw.M4uhYka1yjkPXI7TXYAIHoc1bAMl16bHMa5Rel1JyiM; autoruuid=g5fcbae7b2rh31h6p5i54qkso13bmhrs.d64fdb1048094ade6653bb8d77bfc82f; autoru_gdpr=1; suid=bc138c1d5a6554d5f028f6fec11a91da.a9ca18211fb27ebefea2fe04a11546b6; from=direct; yuidcs=1; crookie=aAs/ROiBsnuk/909OZNBhxIcXPqhyXpNRRJ0N6Sk1kI/Eaknz7W1f3rW2MUp05I5lp/zbVBPKBkNgTKZqk8ULF8Q0Vg=; cmtchd=MTYwNzI1MDk3ODU4MQ==; popup_new_user=new; proven_owner_popup=1; bltsr=1; yuidlt=1; yandexuid=1370723801540629363; X-Vertis-DC=vla; listing_view_session={}; listing_view=%7B%22output_type%22%3A%22models_list%22%2C%22version%22%3A1%7D; new-controls-notify-hidden=true; from_lifetime=1607289350518',
    'Host': 'auto.ru',
    'origin': 'https://auto.ru/cars/all/',
    'Referer': 'https://auto.ru/cars/all/',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0',
    'x-client-app-version': '202012.04.170215',
    'x-client-date': '1607289359144',
    'x-csrf-token': 'd8029e40b7e75f491e05b9191bce2b0854cb8560d32b9010',
    'x-page-request-id': '52d78c631a6cdbf362cebe6be367d327',
    'x-requested-with': 'fetch'
}

In [ ]:
# сделаем тестовый вызов запроса для BMW модели M6 и выведем его на экран, чтобы по нему можно было сделать функцию для создания 23 необходимых столбцов


# но сначала установим главные параметры запроса
mark = 'BMW'  # марка
model = 'M6'  # модель
geo_radius = 200  # радиус вокруг москвы = 200 км
geo_id = [213]  # Москва
top_days = "1"  # кол-во дней от текущей даты назад за которые запрашиваем объявления 

req_body = {
                'catalog_filter' : [{"mark": mark, "model": model}],
                'section': "all",
                'category': "cars",
                'sort': "fresh_relevance_1-desc",
                "top_days": "900",
                "geo_radius":geo_radius,
                "geo_id":geo_id,
                'page': 1
            }

response = requests.post(url, json=req_body, headers = HEADERS)
data = response.json()
for elem in data['offers']:
    print(elem)
    break

In [ ]:
dict_of_num_cars_of_models ={'AC':2, 'Acura':94, 'Alfa Romeo':65, 'Alpina':10, 
                             'AMC':2, 'Ariel':1, 'Asia':1, 'Aston Martin':25, 
                             'Audi':4093, 'Austin Healey':1, 'Bajaj':3, 
                             'Bentley':104, 'BMW':7423, 'Borgward':1, 
                             'Brilliance':10, 'Buick':17, 'BYD':22, 
                             'Cadillac':321, 'Changan':100, 'Chery':593, 
                             'Chevrolet':3670, 'Chrysler':316, 'Citroen':1142, 
                             'Dacia':9, 'Daewoo':1024, 'Daihatsu':47, 
                             'Daimler':5, 'Datsun':212, 'Delage':1, 'Derways':2, 
                             'DKW':1, 'Dodge':413, 'DongFeng':25, 'Doninvest':2, 
                             'DS':3, 'DW Hower':8, 'Eagle':1, 'Excalibur':1, 
                             'FAW':66, 'Ferrari':29, 'Fiat':206, 'Ford':4812, 
                             'Foton':2, 'GAC':14, 'Geely':302, 'Genesis':54, 
                             'GMC':26, 'Great Wall':420, 'Hafei':6, 'Haima':12, 
                             'Haval':110, 'Hawtai':4, 'Heinkel':1, 'Honda':1179, 
                             'HuangHai':1, 'Hummer':70, 'Hyundai':5712, 
                             'Infiniti':1041, 'Innocenti':1, 'Iran Khodro':21, 
                             'Isuzu':43, 'JAC':8, 'Jaguar':308, 'Jeep':466, 'JMC':1, 
                             'Kia':5595, 'LADA (ВАЗ)':8935, 'Lamborghini':14, 
                             'Lancia':8, 'Land Rover':1616, 'Lexus':985, 
                             'Lifan':295, 'Lincoln':62, 'Lotus':1, 'Luxgen':9, 
                             'Mahindra':1, 'Maserati':54, 'Maybach':7, 
                             'Mazda':2311, 'McLaren':5, 'Mercedes-Benz':8005, 
                             'Mercury':16, 'Metrocab':2, 'MG':10, 'MINI':313, 
                             'Mitsubishi':3935, 'Mitsuoka':3, 'Nissan':5776, 
                             'Oldsmobile':10, 'Opel':3131, 'Packard':2, 
                             'Peugeot':1636, 'Plymouth':13, 'Pontiac':38, 
                             'Porsche':714, 'Proton':3, 'PUCH':1, 'Ravon':112, 
                             'Renault':2771, 'Rolls-Royce':55, 'Rover':69, 
                             'Saab':147, 'Saturn':9, 'Scion':9, 'SEAT':77, 
                             'Shanghai Maple':1, 'Skoda':2810, 'Smart':89, 
                             'SsangYong':718, 'Subaru':934, 'Suzuki':878, 
                             'Tatra':5, 'Tesla':43, 'Tianma':2, 'Toyota':4648, 
                             'Triumph':1, 'Vauxhall':1, 'Volkswagen':7109, 
                             'Volvo':2069, 'Vortex':91, 'Wartburg':1, 'Willys':2, 
                             'Zotye':15, 'ZX':4, 'ГАЗ':935, 'Гоночный болид':2, 
                             'ЗАЗ':231, 'ЗИЛ':2, 'ИЖ':129, 'ЛуАЗ':45, 
                             'Москвич':198, 'СМЗ':4, 'ТагАЗ':68, 'УАЗ':1107}

In [ ]:
#list_of_relevant_marks = list(dict_of_num_cars_of_models.keys())